In [1]:
import pandas as pd
import numpy as np
import os
import json
import csv
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
import sys

#csv.field_size_limit(1000000000000000000)

In [2]:
path = "/home/jackalhan/Development/app_data/tf_idf_hiarchical"
dficf_path = os.path.join(path, 'dficf')
tfidf_path = os.path.join(path, 'tfidf')

In [3]:
## pandas style
# dficf_train_df = pd.read_csv(os.path.join(dficf_path, 'dficf_train_all_in_one.csv'), skipinitialspace=True, 
#                              converters={'vector':lambda x: json.loads(x)})
# dficf_test_df = pd.read_csv(os.path.join(dficf_path, 'dficf_test_all_in_one.csv'))

# tfidf_train_df = pd.read_csv(os.path.join(tfidf_path, 'tfidf_train_all_in_one.csv'))
# tfidf_test_df = pd.read_csv(os.path.join(tfidf_path, 'tfidf_test_all_in_one.csv'))

In [4]:
#DFICF Train
with open(os.path.join(dficf_path, 'dficf_train_all_in_one.csv'), 'r') as f:
    reader = csv.reader(f, skipinitialspace=True)
    header = next(reader)
    _X_train = []
    _y_train = []
    for x in reader:
        _X_train.append(json.loads(x[1]))
        _y_train.append(x[2])
    df_X_train = np.array(_X_train)
    df_y_train = np.array(_y_train).reshape(len(_y_train),1)
    del _X_train
    del _y_train
    
#DFICF Test
with open(os.path.join(dficf_path, 'dficf_test_all_in_one.csv'), 'r') as f:
    reader = csv.reader(f, skipinitialspace=True)
    header = next(reader)
    # ------------------------------------------------
    # reader needs to be sorted by actual index value in order to group same documents 
    # ------------------------------------------------
    sorted_reader = sorted(reader, key=lambda row: int(row[4]))
    
    _X_test = []
    _y_test = []
    _potential_category_test = []
    _actual_index_test = []
    for x in sorted_reader:
        _X_test.append(json.loads(x[1]))
        _potential_category_test.append(x[3])
        _actual_index_test.append(int(x[4]))
        _y_test.append(x[5])
    
    df_X_test = np.array(_X_test)
    df_y_test = np.array(_y_test).reshape(len(_y_test),1)
    df_potential_category_test = np.array(_potential_category_test).reshape(len(_potential_category_test),1)
    df_actual_index_test = np.array(_actual_index_test).reshape(len(_actual_index_test),1)
    ## ----------------------------------------------------
    # Actual_index is going to be used to group them so that each index can be 
    # trained to get
    # the best result and compare it with the actual y_test
    ## ----------------------------------------------------
    del _X_test
    del _y_test
    del _potential_category_test
    del _actual_index_test
    
#TFIDF Train
with open(os.path.join(tfidf_path, 'tfidf_train_all_in_one.csv'), 'r') as f:
    reader = csv.reader(f, skipinitialspace=True)
    header = next(reader)
    _X_train = []
    _y_train = []
    for x in reader:
        _X_train.append(json.loads(x[1]))
        _y_train.append(x[2])
    tf_X_train = np.array(_X_train)
    tf_y_train = np.array(_y_train).reshape(len(_y_train),1)
    del _X_train
    del _y_train
        
#TFIDF Test
with open(os.path.join(tfidf_path, 'tfidf_test_all_in_one.csv'), 'r') as f:
    reader = csv.reader(f, skipinitialspace=True)
    header = next(reader)
    _X_test = []
    _y_test = []
    for x in reader:
        _X_test.append(json.loads(x[1]))
        _y_test.append(x[2])
    tf_X_test = np.array(_X_test)
    tf_y_test = np.array(_y_test).reshape(len(_y_test),1)
    del _X_test
    del _y_test

## Preprocessing for TFIDF

In [5]:
# TFIDF
tf_y_unique_labels = np.unique(tf_y_train)
tf_le = preprocessing.LabelEncoder()
tf_le.fit(tf_y_unique_labels)

LabelEncoder()

In [6]:
tf_y_train_encoded = tf_le.transform(tf_y_train)

/home/jackalhan/Development/Projects/Github/py3_machine_learning/venv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
tf_y_test_encoded = tf_le.transform(tf_y_test)

/home/jackalhan/Development/Projects/Github/py3_machine_learning/venv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Preprocessing for DFICF

In [8]:
# TFIDF
df_y_unique_labels = np.unique(df_y_train)
df_le = preprocessing.LabelEncoder()
df_le.fit(df_y_unique_labels)

LabelEncoder()

In [9]:
df_y_train_encoded = df_le.transform(df_y_train)

/home/jackalhan/Development/Projects/Github/py3_machine_learning/venv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
df_y_test_encoded = df_le.transform(df_y_test)

/home/jackalhan/Development/Projects/Github/py3_machine_learning/venv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# K-NearestNeighbours Classifier

In [11]:
clf = CalibratedClassifierCV(base_estimator=LinearSVC(penalty='l2', dual=False), cv=5)

## TFIDF K-NN Processing

In [12]:
clf.fit(tf_X_train, tf_y_train_encoded)

CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv=5, method='sigmoid')

In [13]:
#KNN Labels
clf.classes_

array([0, 1, 2, 3, 4])

In [14]:
#KNN Encoded Labels
tf_y_unique_labels

array(['business', 'entertainment', 'politics', 'sport', 'tech'], 
      dtype='<U13')

In [15]:
correct_predictions = 0
wrong_predictions = 0
for indx, _ in enumerate(tf_X_test):
    tf_X_sample_record = _.reshape(1, _.shape[0])
    #sample_indx = 0
    #tf_X_sample = tf_X_test[sample_indx].reshape(1, tf_X_test[sample_indx].shape[0])
    predicted = clf.predict_proba(tf_X_sample_record)
    print('Actual:', tf_y_test[indx])
    print('Predicted:', predicted)
    predicted_index = np.argmax(predicted)
    print('Predicted Index:', tf_y_test[indx])
    
    if (tf_y_unique_labels[predicted_index] == tf_y_test[indx]):
        correct_predictions +=1
        print('Correct')
    else:
        wrong_predictions +=1
        #print(predicted)

Actual: ['sport']
Predicted: [[ 0.00415192  0.00563792  0.00742273  0.96955817  0.01322926]]
Predicted Index: ['sport']
Correct
Actual: ['politics']
Predicted: [[ 0.00326704  0.00282887  0.98454763  0.00476485  0.00459161]]
Predicted Index: ['politics']
Correct
Actual: ['politics']
Predicted: [[ 0.01078476  0.00289071  0.97757755  0.0018052   0.00694177]]
Predicted Index: ['politics']
Correct
Actual: ['politics']
Predicted: [[ 0.4295376   0.0120849   0.54712496  0.00875847  0.00249408]]
Predicted Index: ['politics']
Correct
Actual: ['entertainment']
Predicted: [[ 0.09437019  0.77492784  0.02954036  0.00973494  0.09142667]]
Predicted Index: ['entertainment']
Correct
Actual: ['entertainment']
Predicted: [[ 0.01451762  0.91217953  0.05197457  0.01501665  0.00631163]]
Predicted Index: ['entertainment']
Correct
Actual: ['entertainment']
Predicted: [[ 0.09561923  0.87823979  0.00272209  0.00354385  0.01987505]]
Predicted Index: ['entertainment']
Correct
Actual: ['tech']
Predicted: [[ 0.00779

Actual: ['business']
Predicted: [[ 0.92793752  0.00753356  0.02573169  0.0049736   0.03382363]]
Predicted Index: ['business']
Correct
Actual: ['sport']
Predicted: [[ 0.01110242  0.00953674  0.01113633  0.95111325  0.01711126]]
Predicted Index: ['sport']
Correct
Actual: ['business']
Predicted: [[ 0.96652914  0.00902714  0.00221783  0.01110223  0.01112367]]
Predicted Index: ['business']
Correct
Actual: ['entertainment']
Predicted: [[ 0.00537911  0.96110287  0.00415371  0.00452684  0.02483747]]
Predicted Index: ['entertainment']
Correct
Actual: ['sport']
Predicted: [[ 0.00549927  0.02166092  0.00522123  0.95985132  0.00776726]]
Predicted Index: ['sport']
Correct
Actual: ['business']
Predicted: [[ 0.97053529  0.00353798  0.0125226   0.00297063  0.0104335 ]]
Predicted Index: ['business']
Correct
Actual: ['business']
Predicted: [[ 0.95778723  0.0147001   0.00956024  0.00603633  0.0119161 ]]
Predicted Index: ['business']
Correct
Actual: ['entertainment']
Predicted: [[ 0.01142939  0.9620553   

Actual: ['business']
Predicted: [[ 0.84750502  0.10460745  0.02522071  0.01498247  0.00768435]]
Predicted Index: ['business']
Correct
Actual: ['business']
Predicted: [[ 0.7541838   0.01776805  0.18678719  0.02357011  0.01769084]]
Predicted Index: ['business']
Correct
Actual: ['business']
Predicted: [[ 0.91597771  0.02203819  0.0441931   0.00787436  0.00991663]]
Predicted Index: ['business']
Correct
Actual: ['business']
Predicted: [[ 0.43216069  0.32067049  0.08408331  0.0242788   0.13880671]]
Predicted Index: ['business']
Correct
Actual: ['sport']
Predicted: [[ 0.00624881  0.01393558  0.00860295  0.95841381  0.01279883]]
Predicted Index: ['sport']
Correct
Actual: ['entertainment']
Predicted: [[ 0.00381253  0.47696396  0.00440773  0.49186628  0.0229495 ]]
Predicted Index: ['entertainment']
Actual: ['business']
Predicted: [[ 0.95152635  0.01370042  0.00401096  0.00185904  0.02890322]]
Predicted Index: ['business']
Correct
Actual: ['tech']
Predicted: [[ 0.00644076  0.14307355  0.0237642  

Actual: ['politics']
Predicted: [[ 0.00640494  0.00930446  0.93396941  0.00658966  0.04373153]]
Predicted Index: ['politics']
Correct
Actual: ['tech']
Predicted: [[ 0.00644678  0.00389311  0.0067509   0.00379258  0.97911664]]
Predicted Index: ['tech']
Correct
Actual: ['politics']
Predicted: [[ 0.02061072  0.0171813   0.94073087  0.01003061  0.01144649]]
Predicted Index: ['politics']
Correct
Actual: ['entertainment']
Predicted: [[ 0.38110712  0.5655239   0.01209278  0.00217491  0.03910129]]
Predicted Index: ['entertainment']
Correct
Actual: ['politics']
Predicted: [[ 0.00376565  0.00579984  0.98564428  0.00141166  0.00337857]]
Predicted Index: ['politics']
Correct
Actual: ['sport']
Predicted: [[ 0.00981429  0.0135122   0.05593727  0.90581814  0.01491809]]
Predicted Index: ['sport']
Correct
Actual: ['politics']
Predicted: [[ 0.00567607  0.02342745  0.95202728  0.00321284  0.01565637]]
Predicted Index: ['politics']
Correct
Actual: ['entertainment']
Predicted: [[ 0.00666641  0.96658234  0.

### TFIDF K-NN Results:

In [16]:
print('Total Test Records : {}'.format(tf_X_test.shape[0]))
print('Total Correct Predictions : {}'.format(correct_predictions))
print('Total Wrong Predictions : {}'.format(wrong_predictions))
tf_knn_rate = (correct_predictions/tf_X_test.shape[0])* 100
print('Success Rate : {}%'.format(tf_knn_rate))

Total Test Records : 295
Total Correct Predictions : 283
Total Wrong Predictions : 12
Success Rate : 95.9322033898305%


## DFICF K-NN Processing

In [17]:
clf.fit(df_X_train, df_y_train_encoded)

CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv=5, method='sigmoid')

In [18]:
#KNN Labels
clf.classes_

array([0, 1, 2, 3, 4])

In [19]:
#KNN Encoded Labels
df_y_unique_labels

array(['business', 'entertainment', 'politics', 'sport', 'tech'], 
      dtype='<U13')

In [45]:
correct_predictions = 0
wrong_predictions = 0
window_size = df_y_unique_labels.shape[0]
# iteration should be achieved in every jump_step/block like window_size, 
# because, df_X_test is ordered by the document index, or in other terms same document
# same document is getting repearted in each window
# thus, in each window, we need to iterate again for each document in this window.

from collections import Counter
import operator

for step in range(0, df_X_test.shape[0], window_size):
    max_predicted = None
    max_predicted_value = 0
    print(20 * '*')
    print(20 * '*')
    category_index_prediction = {}
    category_index_occurences = {}
    for indx in range(step, step + window_size):
        print('step:', step)
        print('index:', indx)
        print('record:', df_X_test[indx])
        print('label:', df_potential_category_test[indx])
        df_X_sample_record = df_X_test[indx].reshape(1, df_X_test[indx].shape[0])
        #sample_indx = 0
        #tf_X_sample = tf_X_test[sample_indx].reshape(1, tf_X_test[sample_indx].shape[0])
        predicted = clf.predict_proba(df_X_sample_record)
        predicted_value = np.max(predicted)
        print('Predictions in Window:', predicted)
        #average pooling
        predicted_index = np.argmax(predicted)
        try:
            occurence_val = category_index_occurences[predicted_index]
            occurence_val += 1
            category_index_occurences[predicted_index] = occurence_val
            average_val = category_index_prediction[predicted_index]
            average_val += predicted_value
            #new_avg_value = average_val / occurence_val
            category_index_prediction[predicted_index] = average_val
        except:
            category_index_occurences[predicted_index] = 1
            category_index_prediction[predicted_index] = predicted_value
    
    print('occ:', category_index_occurences)
    print('pred:', category_index_prediction)
    max_predicted_index = max(category_index_prediction.items(), key=operator.itemgetter(1))[0]
    #max_predicted = 
    #max predictions
#         if predicted_value > max_predicted_value:
#             max_predicted_value = predicted_value
#             max_predicted = predicted
    
#     print('Max predicted:', max_predicted)
#     print('Max predicted value:', max_predicted_value)

   
    #max_predicted_index = np.argmax(max_predicted)
#     print('Max predicted index:', max_predicted_index)
    
#     print('Predicted:', tf_y_unique_labels[max_predicted_index])
#     print('Actual:', df_y_test[step])
    if (df_y_unique_labels[max_predicted_index] == df_y_test[step]):
        correct_predictions +=1
        print('OK!')
    else:
        wrong_predictions +=1
        print('Max predicted:', max_predicted)
        print('Max predicted value:', max_predicted_value)
        print('Max predicted index:', max_predicted_index)
        print('Predicted:', tf_y_unique_labels[max_predicted_index])
        print('Actual:', df_y_test[step])
        print('Wrong','!'* 10)

********************
********************
step: 0
index: 0
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['tech']
Predictions in Window: [[ 0.19264634  0.15204516  0.07325753  0.52873046  0.0533205 ]]
step: 0
index: 1
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.43137376  0.15929349  0.07447652  0.28881979  0.04603644]]
step: 0
index: 2
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['politics']
Predictions in Window: [[ 0.17599861  0.15166349  0.08110303  0.54828406  0.04295081]]
step: 0
index: 3
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['sport']
Predictions in Window: [[ 0.01413053  0.02931443  0.03631769  0.89606741  0.02416994]]
step: 0
index: 4
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.13489101  0.19146862  0.06751527  0.56688882  0.03923628]]
occ: {0: 1, 3: 4}
pred: {0: 0.4313737573840169, 3: 2.5399707515608716}
OK!
********************
********************
step: 5
index: 5
record: [ 0

Predictions in Window: [[ 0.41474301  0.18746147  0.10061061  0.23330834  0.06387658]]
step: 60
index: 62
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['sport']
Predictions in Window: [[ 0.31500522  0.2147926   0.09518486  0.32116056  0.05385677]]
step: 60
index: 63
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.87111231  0.03681371  0.02929862  0.03983841  0.02293697]]
step: 60
index: 64
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.37029554  0.2604695   0.08825354  0.22302778  0.05795364]]
occ: {0: 4, 3: 1}
pred: {0: 2.37699737529854, 3: 0.32116055508057312}
OK!
********************
********************
step: 65
index: 65
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.8390762   0.06039592  0.03781266  0.03432187  0.02839335]]
step: 65
index: 66
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.40538891  0.32720012  0.

label: ['sport']
Predictions in Window: [[ 0.3197884   0.19037778  0.08776129  0.34909238  0.05298015]]
step: 115
index: 116
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['tech']
Predictions in Window: [[ 0.51203935  0.14644984  0.08609065  0.20195127  0.05346889]]
step: 115
index: 117
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['politics']
Predictions in Window: [[ 0.62237807  0.10846457  0.08197615  0.14659484  0.04058637]]
step: 115
index: 118
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.38540634  0.21551677  0.08972981  0.25510082  0.05424626]]
step: 115
index: 119
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.93787791  0.01494122  0.01599105  0.01316696  0.01802286]]
occ: {0: 4, 3: 1}
pred: {0: 2.4577016732869685, 3: 0.34909237684889749}
OK!
********************
********************
step: 120
index: 120
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['sport']
Predictions in Window: [[ 0.3046

Predictions in Window: [[ 0.08923735  0.27760323  0.06585341  0.52593666  0.04136935]]
occ: {1: 4, 3: 1}
pred: {1: 2.3907257785067513, 3: 0.52593665969636394}
OK!
********************
********************
step: 170
index: 170
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['tech']
Predictions in Window: [[ 0.20522558  0.17096059  0.08158627  0.49412329  0.04810427]]
step: 170
index: 171
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['sport']
Predictions in Window: [[ 0.03695856  0.06617159  0.0497675   0.81927028  0.02783208]]
step: 170
index: 172
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.29101548  0.20526574  0.08167335  0.37509882  0.04694661]]
step: 170
index: 173
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.15411604  0.16660951  0.07136426  0.5694266   0.03848359]]
step: 170
index: 174
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['politics']
Predictions in Window: [[ 0.17744626  0.16608408 

Predictions in Window: [[ 0.08159234  0.17814068  0.04090581  0.05013595  0.64922522]]
step: 220
index: 223
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['sport']
Predictions in Window: [[ 0.18696213  0.26929162  0.07981388  0.41077377  0.05315859]]
step: 220
index: 224
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.21451102  0.45183381  0.07642314  0.19119821  0.06603382]]
occ: {0: 1, 1: 2, 3: 1, 4: 1}
pred: {0: 0.46030702038286664, 1: 0.83421228483167953, 3: 0.41077377103112844, 4: 0.6492252160969112}
Max predicted: None
Max predicted value: 0
Max predicted index: 1
Predicted: entertainment
Actual: ['tech']
Wrong !!!!!!!!!!
********************
********************
step: 225
index: 225
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[  9.66461475e-01   1.31543991e-02   7.36215936e-03   6.83447250e-04
    1.23385188e-02]]
step: 225
index: 226
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['politics']
Pred

...

Predictions in Window: [[ 0.20339592  0.22575739  0.08486315  0.44177311  0.04421043]]
occ: {2: 1, 3: 4}
pred: {2: 0.71616994636640419, 3: 1.8151934091215891}
Max predicted: None
Max predicted value: 0
Max predicted index: 3
Predicted: sport
Actual: ['politics']
Wrong !!!!!!!!!!
********************
********************
step: 1245
index: 1245
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['tech']
Predictions in Window: [[ 0.45069806  0.18142114  0.06508478  0.24252926  0.06026676]]
step: 1245
index: 1246
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.31949377  0.25838454  0.07349226  0.30142734  0.0472021 ]]
step: 1245
index: 1247
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.90586928  0.02554542  0.02167023  0.02515487  0.0217602 ]]
step: 1245
index: 1248
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['sport']
Predictions in Window: [[ 0.24506319  0.23622551  0.07684978  0.39656942  0.0452921 ]]
step

Predictions in Window: [[ 0.87372362  0.05779239  0.03696163  0.00774069  0.02378166]]
step: 1295
index: 1298
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.64682221  0.18943879  0.08057971  0.03670993  0.04644936]]
step: 1295
index: 1299
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['sport']
Predictions in Window: [[ 0.5918466   0.15918137  0.09429622  0.10380294  0.05087287]]
occ: {0: 5}
pred: {0: 3.449858702733084}
OK!
********************
********************
step: 1300
index: 1300
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['tech']
Predictions in Window: [[ 0.21302183  0.16609749  0.28372527  0.24565605  0.09149936]]
step: 1300
index: 1301
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['politics']
Predictions in Window: [[ 0.00286475  0.01538973  0.94986863  0.01710233  0.01477455]]
step: 1300
index: 1302
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.27541236  0.18185829  0.22731801  0.243496

record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['sport']
Predictions in Window: [[ 0.2697555   0.19653893  0.15064206  0.27780666  0.10525684]]
step: 1345
index: 1347
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.27830045  0.23202864  0.24143219  0.12314149  0.12509723]]
step: 1345
index: 1348
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.38707549  0.11600476  0.36596415  0.03759146  0.09336414]]
step: 1345
index: 1349
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['tech']
Predictions in Window: [[ 0.16215958  0.12851379  0.38992004  0.04670237  0.27270421]]
occ: {0: 2, 2: 2, 3: 1}
pred: {0: 0.66537593955239527, 2: 1.3745057928236069, 3: 0.27780666180993852}
OK!
********************
********************
step: 1350
index: 1350
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.34287349  0.23085534  0.09216742  0.2836996   0.05040415]]
step: 1350
index: 1351
record: [

Predictions in Window: [[ 0.51355606  0.15771045  0.09538211  0.17107673  0.06227465]]
step: 1410
index: 1412
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['politics']
Predictions in Window: [[ 0.44771488  0.02320524  0.48958587  0.00938269  0.03011132]]
step: 1410
index: 1413
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[  9.46707051e-01   3.04351178e-03   3.09365540e-02   4.32340283e-04
    1.88805429e-02]]
step: 1410
index: 1414
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.73228032  0.13992134  0.05212531  0.02740406  0.04826897]]
occ: {0: 4, 2: 1}
pred: {0: 3.0820074068119432, 2: 0.48958587138720178}
OK!
********************
********************
step: 1415
index: 1415
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.62090865  0.16452718  0.0746662   0.09027016  0.04962781]]
step: 1415
index: 1416
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['tech']
Pred

record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['entertainment']
Predictions in Window: [[ 0.42523026  0.28981328  0.06304795  0.1735124   0.04839611]]
occ: {0: 4, 3: 1}
pred: {0: 1.6722322396865728, 3: 0.7791622890809714}
Max predicted: None
Max predicted value: 0
Max predicted index: 0
Predicted: business
Actual: ['sport']
Wrong !!!!!!!!!!
********************
********************
step: 1465
index: 1465
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['business']
Predictions in Window: [[ 0.25037228  0.22510107  0.07857922  0.39696851  0.04897892]]
step: 1465
index: 1466
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['politics']
Predictions in Window: [[ 0.17317721  0.19800861  0.07982935  0.50219348  0.04679134]]
step: 1465
index: 1467
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['tech']
Predictions in Window: [[ 0.17545093  0.20141224  0.0752015   0.49968621  0.04824912]]
step: 1465
index: 1468
record: [ 0.  0.  0. ...,  0.  0.  0.]
label: ['sport']
Predictions in Window: [[ 0.06238

### DFICF K-NN Results: 

In [46]:
print('Total Test Records : {}'.format(df_X_test.shape[0]/window_size))
print('Total Correct Predictions : {}'.format(correct_predictions))
print('Total Wrong Predictions : {}'.format(wrong_predictions))
df_knn_rate = (correct_predictions/(df_X_test.shape[0]/window_size))* 100
print('Success Rate : {}%'.format(df_knn_rate))

Total Test Records : 295.0
Total Correct Predictions : 202
Total Wrong Predictions : 93
Success Rate : 68.47457627118644%
